# Code adapted from https://github.com/scjones5/nyt-bestsellers

In [1]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 100)

In [2]:
r = requests.get("https://api.nytimes.com/svc/books/v3/lists/2019-08-11/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH")

In [3]:
bs = r.json()

In [4]:
for key in bs.keys():
    print(key)

status
copyright
num_results
last_modified
results


In [5]:
bs = json_normalize(bs["results"]["books"])

/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_54512/577837607.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  bs = json_normalize(bs["results"]["books"])


In [6]:
bs.head()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,1,0,0,1538750554,9781538750551,Grand Central,A World War II veteran on parole must find the...,0.00,ONE GOOD DEED,David Baldacci,by David Baldacci,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/One-Good-Deed-David-Bal...,,,,,,"[{'isbn10': '1538750562', 'isbn13': '978153875...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/f98ddab9-e895-565a-818a-b4488ec8c2e4
1,2,2,46,0,0,0735219095,9780735219090,Putnam,In a quiet town on the North Carolina coast in...,0.00,WHERE THE CRAWDADS SING,Delia Owens,by Delia Owens,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Where-Crawdads-Sing-Del...,,,,,,"[{'isbn10': '0735219095', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/053b4109-4555-5aa1-9b39-cc40549bcdf0
2,3,1,2,0,0,0062834894,9780062834898,Harper,"Gabriel Allon, the chief of Israeli intelligen...",0.00,THE NEW GIRL,Daniel Silva,by Daniel Silva,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/New-Girl-Novel-Gabriel-...,,,,,,"[{'isbn10': '0062834835', 'isbn13': '978006283...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/e308e25d-b1dd-559d-92b2-ec81b7bbad80
3,4,3,2,0,0,0385537077,9780385537070,Doubleday,Two boys respond to horrors at a Jim Crow-era ...,0.00,THE NICKEL BOYS,Colson Whitehead,by Colson Whitehead,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Nickel-Boys-Novel-Colso...,,https://www.nytimes.com/2019/07/11/books/revie...,,,,"[{'isbn10': '0385537077', 'isbn13': '978038553...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/b9c84bf7-ad1c-56a3-9b42-e9b37e488f42
4,5,8,24,0,0,0525620931,9780525620938,Doubleday,A decorated World War II veteran shoots and ki...,0.00,THE RECKONING,John Grisham,by John Grisham,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/Reckoning-Novel-John-Gr...,,,,,,"[{'isbn10': '0385544154', 'isbn13': '978038554...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/821816fe-b672-571b-8cc7-5a23a25b58e8


In [7]:
bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   rank                  15 non-null     int64 
 1   rank_last_week        15 non-null     int64 
 2   weeks_on_list         15 non-null     int64 
 3   asterisk              15 non-null     int64 
 4   dagger                15 non-null     int64 
 5   primary_isbn10        15 non-null     object
 6   primary_isbn13        15 non-null     object
 7   publisher             15 non-null     object
 8   description           15 non-null     object
 9   price                 15 non-null     object
 10  title                 15 non-null     object
 11  author                15 non-null     object
 12  contributor           15 non-null     object
 13  contributor_note      15 non-null     object
 14  book_image            15 non-null     object
 15  book_image_width      15 non-null     int6

In [12]:
def parse_books(book_list, date):
    '''
    Based on: function parse_articles from
    This function takes in a response to the NYT books api and parses
    the books into a list of dictionaries
    '''
    books = []
    for i in book_list['results']['books']:
        dic = {}
        dic['age_group'] = i['age_group']
        dic['author'] = i['author'].encode("utf8")
        dic['book_review_link'] = i['book_review_link'].encode("utf8")
        dic['contributor'] = i['contributor'].encode("utf8")
        dic['dagger'] = i['dagger']
        dic['first_chapter_link'] = i['first_chapter_link'].encode("utf8")
        dic['price'] = i['price']
        dic['primary_isbn10'] = i['primary_isbn10'].encode("utf8")
        dic['primary_isbn13'] = i['primary_isbn13'].encode("utf8")
        dic['publisher'] = i['publisher'].encode("utf8")
        dic['sunday_review_link'] = i['sunday_review_link'].encode("utf8")
        dic['title'] = i['title'].encode("utf8")
        dic['weeks_on_list'] = i['weeks_on_list']
        dic['date'] = date  # add a date so that we know when it was a best-seller
        books.append(dic)
    return(books)


In [28]:
#sample request: https://api.nytimes.com/svc/books/v3/lists/2017-03-13/Combined%20Print%20and%20E-Book%20Fiction.json?api_key=<your_api_key>
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "combined-print-and-e-book-fiction"   #you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = "LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH"


bestsellers = []
weeks = []
num_calls_to_make = 550 # you can use up to 1000 api calls a day so max=1000
ctr = 0

# lists update once a week, every Sunday
for i in range(0,num_calls_to_make):
    today = datetime.datetime.today()
    idx = (today.weekday() + 1) % 7
    last_sun = today - datetime.timedelta((7*ctr)+idx)
    last_sun = last_sun.date()
    print(last_sun)
    url = "%s/%s/%s?api-key=%s" % (BOOKS_ROOT, last_sun, LIST, API_KEY)
    url = url.strip()
    print(url)
    r = requests.get(url)
    if (r.status_code == 200):
        results = r.json()
        books = json_normalize(results["results"]["books"]) #parse_books(results, date)
        bestsellers.append(books)
        weeks.append(str(last_sun))
    else:
        print("API call did not work!")
        print(r)
        #break
    ctr = ctr + 1
    time.sleep(6)
#     if (bestsellers[0].keys() is not None):
#        keys = bestsellers[0].keys()
#        # Print all results to a csv file
#        with open('bestsellers.csv', 'wb') as output_file:
#            dict_writer = csv.DictWriter(output_file, keys)
#            dict_writer.writeheader()
#            dict_writer.writerows(bestsellers)

2022-06-12
https://api.nytimes.com/svc/books/v3/lists/2022-06-12/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH


/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_54512/592614727.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  books = json_normalize(results["results"]["books"]) #parse_books(results, date)


2022-06-05
https://api.nytimes.com/svc/books/v3/lists/2022-06-05/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-29
https://api.nytimes.com/svc/books/v3/lists/2022-05-29/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-22
https://api.nytimes.com/svc/books/v3/lists/2022-05-22/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-15
https://api.nytimes.com/svc/books/v3/lists/2022-05-15/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-08
https://api.nytimes.com/svc/books/v3/lists/2022-05-08/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-01
https://api.nytimes.com/svc/books/v3/lists/2022-05-01/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-04-24
https://api.nytimes.com/svc/books/v3/lists/2022-04-24/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-04-17
https://a

2021-04-18
https://api.nytimes.com/svc/books/v3/lists/2021-04-18/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-04-11
https://api.nytimes.com/svc/books/v3/lists/2021-04-11/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-04-04
https://api.nytimes.com/svc/books/v3/lists/2021-04-04/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-28
https://api.nytimes.com/svc/books/v3/lists/2021-03-28/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-21
https://api.nytimes.com/svc/books/v3/lists/2021-03-21/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-14
https://api.nytimes.com/svc/books/v3/lists/2021-03-14/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-07
https://api.nytimes.com/svc/books/v3/lists/2021-03-07/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-02-28
https://a

2020-03-01
https://api.nytimes.com/svc/books/v3/lists/2020-03-01/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-02-23
https://api.nytimes.com/svc/books/v3/lists/2020-02-23/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-02-16
https://api.nytimes.com/svc/books/v3/lists/2020-02-16/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-02-09
https://api.nytimes.com/svc/books/v3/lists/2020-02-09/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-02-02
https://api.nytimes.com/svc/books/v3/lists/2020-02-02/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-26
https://api.nytimes.com/svc/books/v3/lists/2020-01-26/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-19
https://api.nytimes.com/svc/books/v3/lists/2020-01-19/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-12
https://a

...

2016-10-09
https://api.nytimes.com/svc/books/v3/lists/2016-10-09/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-10-02
https://api.nytimes.com/svc/books/v3/lists/2016-10-02/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-09-25
https://api.nytimes.com/svc/books/v3/lists/2016-09-25/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-09-18
https://api.nytimes.com/svc/books/v3/lists/2016-09-18/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-09-11
https://api.nytimes.com/svc/books/v3/lists/2016-09-11/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-09-04
https://api.nytimes.com/svc/books/v3/lists/2016-09-04/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-08-28
https://api.nytimes.com/svc/books/v3/lists/2016-08-28/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-08-21
https://a

2015-08-23
https://api.nytimes.com/svc/books/v3/lists/2015-08-23/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2015-08-16
https://api.nytimes.com/svc/books/v3/lists/2015-08-16/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2015-08-09
https://api.nytimes.com/svc/books/v3/lists/2015-08-09/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2015-08-02
https://api.nytimes.com/svc/books/v3/lists/2015-08-02/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2015-07-26
https://api.nytimes.com/svc/books/v3/lists/2015-07-26/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2015-07-19
https://api.nytimes.com/svc/books/v3/lists/2015-07-19/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2015-07-12
https://api.nytimes.com/svc/books/v3/lists/2015-07-12/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2015-07-05
https://a

2014-07-06
https://api.nytimes.com/svc/books/v3/lists/2014-07-06/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2014-06-29
https://api.nytimes.com/svc/books/v3/lists/2014-06-29/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2014-06-22
https://api.nytimes.com/svc/books/v3/lists/2014-06-22/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2014-06-15
https://api.nytimes.com/svc/books/v3/lists/2014-06-15/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2014-06-08
https://api.nytimes.com/svc/books/v3/lists/2014-06-08/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2014-06-01
https://api.nytimes.com/svc/books/v3/lists/2014-06-01/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2014-05-25
https://api.nytimes.com/svc/books/v3/lists/2014-05-25/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2014-05-18
https://a

2013-05-19
https://api.nytimes.com/svc/books/v3/lists/2013-05-19/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2013-05-12
https://api.nytimes.com/svc/books/v3/lists/2013-05-12/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2013-05-05
https://api.nytimes.com/svc/books/v3/lists/2013-05-05/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2013-04-28
https://api.nytimes.com/svc/books/v3/lists/2013-04-28/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2013-04-21
https://api.nytimes.com/svc/books/v3/lists/2013-04-21/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2013-04-14
https://api.nytimes.com/svc/books/v3/lists/2013-04-14/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2013-04-07
https://api.nytimes.com/svc/books/v3/lists/2013-04-07/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2013-03-31
https://a

2012-04-01
https://api.nytimes.com/svc/books/v3/lists/2012-04-01/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2012-03-25
https://api.nytimes.com/svc/books/v3/lists/2012-03-25/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2012-03-18
https://api.nytimes.com/svc/books/v3/lists/2012-03-18/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2012-03-11
https://api.nytimes.com/svc/books/v3/lists/2012-03-11/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2012-03-04
https://api.nytimes.com/svc/books/v3/lists/2012-03-04/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2012-02-26
https://api.nytimes.com/svc/books/v3/lists/2012-02-26/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2012-02-19
https://api.nytimes.com/svc/books/v3/lists/2012-02-19/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2012-02-12
https://a

In [44]:
bestsellers_df = pd.concat(bestsellers)

In [29]:
#repeat weeks 15 times to add to df
#https://www.pythonforbeginners.com/lists/repeat-each-element-in-a-list-in-python
k = 15
repeated_weeks = []
for element in weeks:
    for i in range(k):
       repeated_weeks.append(element)

In [33]:
len(weeks)

550

In [32]:
#checking to make sure length of bestsellers matches length of repeated_weeks
len(bestsellers_df)

9600

In [31]:
#checking to make sure length of bestsellers matches length of repeated_weeks
len(repeated_weeks)

8250

In [65]:
bestsellers_df.to_csv("../data/last_ten_years_nyt_bestsellers_all_data.csv", index=False)

In [39]:
bestsellers_df.head(20)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,1,0,0,1250278201,9781250278203,St. Martin's,"Harry Booth, a master thief, breaks things off...",0.00,NIGHTWORK,Nora Roberts,by Nora Roberts,,https://storage.googleapis.com/du-prd/books/im...,329.0,500.0,https://www.amazon.com/dp/1250278198?tag=NYTBS...,,,,,,"[{'isbn10': '1250278198', 'isbn13': '978125027...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/41951f87-0c4d-5227-be44-1ac0ec00616c
1,2,4,162,0,0,0735219109,9780735219106,Putnam,In a quiet town on the North Carolina coast in...,0.00,WHERE THE CRAWDADS SING,Delia Owens,by Delia Owens,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,https://www.amazon.com/Where-Crawdads-Sing-Del...,,,,,,"[{'isbn10': '0735219095', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/08a5730d-43e5-586d-8ce6-4b891eb75fe1
2,3,3,50,0,0,1501110365,9781501110368,Atria,A battered wife raised in a violent home attem...,0.00,IT ENDS WITH US,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,319.0,495.0,http://www.amazon.com/Ends-Us-Novel-Colleen-Ho...,,,,,,"[{'isbn10': '1501110365', 'isbn13': '978150111...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/e2a3545e-e9cb-5828-9d97-50a798a0e4f6
3,4,2,4,0,0,0593334833,9780593334836,Berkley,"While on vacation in North Carolina, a literar...",0.00,BOOK LOVERS,Emily Henry,by Emily Henry,,https://storage.googleapis.com/du-prd/books/im...,332.0,500.0,https://www.amazon.com/dp/0593334833?tag=NYTBS...,,,,,,"[{'isbn10': '0593334833', 'isbn13': '978059333...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/31fc527e-86c7-5743-a111-ea333a630706
4,5,5,25,0,0,1538724731,9781538724736,Grand Central,Lowen Ashleigh is hired by the husband of an i...,0.00,VERITY,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,324.0,500.0,https://www.amazon.com/dp/1791392792?tag=NYTBS...,,,,,,"[{'isbn10': '1791392792', 'isbn13': '978179139...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/013c7729-a1de-5e5c-921a-da43243f9a4a
5,6,6,48,0,0,1501161938,9781501161933,Washington Square/Atria,A movie icon recounts stories of her loves and...,0.00,THE SEVEN HUSBANDS OF EVELYN HUGO,Taylor Jenkins Reid,by Taylor Jenkins Reid,,https://storage.googleapis.com/du-prd/books/im...,315.0,500.0,https://www.amazon.com/dp/1501161938?tag=NYTBS...,,,,,,"[{'isbn10': '1501161938', 'isbn13': '978150116...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/d9511fac-ee44-5a87-9af7-2cd6a6f8f984
6,7,7,20,0,0,1476753180,9781476753188,Atria,"Tate Collins and Miles Archer, an airline pilo...",0.00,UGLY LOVE,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,128.0,198.0,http://www.amazon.com/Ugly-Love-Novel-Colleen-...,,,,,,"[{'isbn10': '1476753180', 'isbn13': '978147675...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/08464a1b-cfd4-5d1b-8898-e206fb95deeb
7,8,0,1,0,0,0593419685,9780593419687,Putnam,The 19th book in the NUMA Files series. A sear...,0.00,CLIVE CUSSLER'S DARK VECTOR,Graham Brown,by Graham Brown,,https://storage.googleapis.com/du-prd/books/im...,331.0,500.0,https://www.amazon.com/dp/0593419677?tag=NYTBS...,,,,,,"[{'isbn10': '0593419677', 'isbn13': '978059341...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/fef45730-117d-573b-83a9-c315f3761bc6
8,9,10,27,0,0,1984806750,9781984806758,Berkley,Opposites Poppy and Alex meet to vacation toge...,0.00,PEOPLE WE MEET ON VACATION,Emily Henry,by Emily Henry,,https://storage.googleapis.com/du-prd/books/im...,333.0,500.0,https://www.amazon.com/dp/1984806750?tag=NYTBS...,,,,,,"[{'isbn10'

In [35]:
bestsellers_df.tail(20)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,1,0,0,None,9780316194495,"Little, Brown & Company",Alex Cross investigates when the president’s c...,0.00,KILL ALEX CROSS,James Patterson,by James Patterson,,https://storage.googleapis.com/du-prd/books/im...,320.0,495.0,http://www.amazon.com/Kill-Alex-Cross-James-Pa...,,,,,,"[{'isbn10': '0316198730', 'isbn13': '978031619...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/70b3e460-3a2f-57d3-b226-635b98302918
1,2,0,1,0,0,None,9781101548134,Penguin Group,"Pursuing a shoplifter, Kinsey Millhone discove...",0.00,V IS FOR VENGEANCE,Sue Grafton,by Sue Grafton,,https://storage.googleapis.com/du-prd/books/im...,327.0,495.0,http://www.amazon.com/V-Vengeance-Sue-Grafton/...,,,,https://www.nytimes.com/2011/11/20/books/revie...,,"[{'isbn10': '0399157867', 'isbn13': '978039915...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/69c3b73d-bebb-5a0b-ba93-9f81cd9ef375
2,3,1,2,0,0,1451627289,9781451627282,Scribner,An English teacher travels back to 1958 by way...,0.00,11/22/63,Stephen King,by Stephen King,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,http://www.amazon.com/11-22-63-A-Novel/dp/1451...,,https://www.nytimes.com/2011/10/31/books/steph...,,https://www.nytimes.com/2011/11/13/books/revie...,,"[{'isbn10': '1451627289', 'isbn13': '978145162...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/99803a1d-43f9-5cde-8625-a1d4d5ebd1a4
3,4,2,4,0,0,None,9780385535250,Knopf Doubleday Publishing,Partners in a small law firm take on a big cas...,0.00,THE LITIGATORS,John Grisham,by John Grisham,,https://storage.googleapis.com/du-prd/books/im...,184.0,280.0,http://www.amazon.com/The-Litigators-John-Gris...,,,,,,"[{'isbn10': '0385535139', 'isbn13': '978038553...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/2763141c-b45e-524d-93fb-3f8d72ccf684
4,5,0,1,0,0,None,9781101545928,Putnam,"In the ninth NUMA Files novel, Kurt Austin and...",0.00,DEVIL’S GATE,Clive Cussler and Graham Brown,by Clive Cussler and Graham Brown,,https://storage.googleapis.com/du-prd/books/im...,264.0,400.0,http://www.amazon.com/Devils-Gate-Numa-Files-B...,,,,,,"[{'isbn10': '0399157824', 'isbn13': '978039915...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/04d1b92c-950f-5aa4-bd6b-bb4d75a231f9
5,6,4,3,0,0,None,9780446573030,Grand Central Publishing,A military investigator uncovers a conspiracy.,0.00,ZERO DAY,David Baldacci,by David Baldacci,,https://storage.googleapis.com/du-prd/books/im...,276.0,495.0,http://www.amazon.com/Zero-Day-David-Baldacci/...,,,,,,"[{'isbn10': '0446573019', 'isbn13': '978044657...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/55bb7e5b-df9d-5537-b9cf-bc15f75c69e3
6,7,5,6,0,0,None,9781455502547,Grand Central Publishing,A man and woman who have gone their separate w...,0.00,THE BEST OF ME,Nicholas Sparks,by Nicholas Sparks,,https://storage.googleapis.com/du-prd/books/im...,307.0,495.0,http://www.amazon.com/The-Best-Me-Nicholas-Spa...,,,,,,[],"[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/ea60d6e1-2b82-518a-ba08-e001285afb5e
7,8,3,3,0,0,0425243214,9780425243213,Penguin Group,"A historic hotel in Boonsboro, Md., is getting...",0.00,THE NEXT ALWAYS,Nora Roberts,by Nora Roberts,,https://storage.googleapis.com/du-prd/books/im...,330.0,487.0,http://www.amazon.com/The-Next-Always-BoonsBor...,,,,,,"[{'isbn10': '0425243214', 'isbn13': '978042524...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/f4eb3f60-9612-50f9-a193-2880aaabf5f2
8,9,6,40,0,0,1440697663,9781440697661,Penguin Group,A young white woman and two black maids in 196...,0.00,THE HELP,Kathryn Stockett,by Kathryn Stockett,,https://storage.g

In [36]:
bestsellers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9600 entries, 0 to 19
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rank                  9600 non-null   int64  
 1   rank_last_week        9600 non-null   int64  
 2   weeks_on_list         9600 non-null   int64  
 3   asterisk              9600 non-null   int64  
 4   dagger                9600 non-null   int64  
 5   primary_isbn10        9600 non-null   object 
 6   primary_isbn13        9600 non-null   object 
 7   publisher             9600 non-null   object 
 8   description           9600 non-null   object 
 9   price                 9600 non-null   object 
 10  title                 9600 non-null   object 
 11  author                9600 non-null   object 
 12  contributor           9600 non-null   object 
 13  contributor_note      9599 non-null   object 
 14  book_image            9590 non-null   object 
 15  book_image_width      9

In [46]:
#after looking at head and tail, it looks like they switched from top 20 to top 15 at some point
#for normalization, let's narrow it down to top 15 across all weeks
#https://www.geeksforgeeks.org/python-pandas-dataframe-isin/
bestsellers_df['rank'] = bestsellers_df['rank'].astype(str)
filter_15 = bestsellers_df['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [47]:
top_15_bestsellers = bestsellers_df[filter_15]

In [48]:
top_15_bestsellers.head()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,1,0,0,1250278201,9781250278203,St. Martin's,"Harry Booth, a master thief, breaks things off...",0.00,NIGHTWORK,Nora Roberts,by Nora Roberts,,https://storage.googleapis.com/du-prd/books/im...,329.0,500.0,https://www.amazon.com/dp/1250278198?tag=NYTBS...,,,,,,"[{'isbn10': '1250278198', 'isbn13': '978125027...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/41951f87-0c4d-5227-be44-1ac0ec00616c
1,2,4,162,0,0,0735219109,9780735219106,Putnam,In a quiet town on the North Carolina coast in...,0.00,WHERE THE CRAWDADS SING,Delia Owens,by Delia Owens,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,https://www.amazon.com/Where-Crawdads-Sing-Del...,,,,,,"[{'isbn10': '0735219095', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/08a5730d-43e5-586d-8ce6-4b891eb75fe1
2,3,3,50,0,0,1501110365,9781501110368,Atria,A battered wife raised in a violent home attem...,0.00,IT ENDS WITH US,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,319.0,495.0,http://www.amazon.com/Ends-Us-Novel-Colleen-Ho...,,,,,,"[{'isbn10': '1501110365', 'isbn13': '978150111...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/e2a3545e-e9cb-5828-9d97-50a798a0e4f6
3,4,2,4,0,0,0593334833,9780593334836,Berkley,"While on vacation in North Carolina, a literar...",0.00,BOOK LOVERS,Emily Henry,by Emily Henry,,https://storage.googleapis.com/du-prd/books/im...,332.0,500.0,https://www.amazon.com/dp/0593334833?tag=NYTBS...,,,,,,"[{'isbn10': '0593334833', 'isbn13': '978059333...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/31fc527e-86c7-5743-a111-ea333a630706
4,5,5,25,0,0,1538724731,9781538724736,Grand Central,Lowen Ashleigh is hired by the husband of an i...,0.00,VERITY,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,324.0,500.0,https://www.amazon.com/dp/1791392792?tag=NYTBS...,,,,,,"[{'isbn10': '1791392792', 'isbn13': '978179139...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/013c7729-a1de-5e5c-921a-da43243f9a4a


In [49]:
#and now it's the same length as repeated_weeks, so it will be easy to add that on!
len(top_15_bestsellers)

8250

In [51]:
#let's go ahead and add a week column
top_15_bestsellers['week'] = repeated_weeks

#and check to make sure it worked correctly
top_15_bestsellers.tail(20)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri,week
10,11,10,15,0,0,0307272117,9780307272119,Knopf Doubleday Publishing,A hacker and a journalist investigate the disa...,0.00,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,by Stieg Larsson,,https://storage.googleapis.com/du-prd/books/im...,277.0,495.0,http://www.amazon.com/Girl-Dragon-Tattoo-Stieg...,,https://www.nytimes.com/2008/09/30/books/30kak...,,https://www.nytimes.com/2008/09/14/books/revie...,,"[{'isbn10': '0307454541', 'isbn13': '978030745...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/61741728-91b7-59e6-b70e-61e4e41c4a1f,2011-12-11
11,12,8,4,0,0,0425243214,9780425243213,Penguin Group,"A historic hotel in Boonsboro, Md., is getting...",0.00,THE NEXT ALWAYS,Nora Roberts,by Nora Roberts,,https://storage.googleapis.com/du-prd/books/im...,330.0,487.0,http://www.amazon.com/The-Next-Always-BoonsBor...,,,,,,"[{'isbn10': '0425243214', 'isbn13': '978042524...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/f4eb3f60-9612-50f9-a193-2880aaabf5f2,2011-12-11
12,13,13,7,0,0,0345527704,9780345527707,Random House Publishing,A killer is after the bounty hunter Stephanie ...,0.00,SMOKIN’ SEVENTEEN,Janet Evanovich,by Janet Evanovich,,https://storage.googleapis.com/du-prd/books/im...,301.0,495.0,http://www.amazon.com/Smokin-Seventeen-Stephan...,,,,,,"[{'isbn10': '0345527682', 'isbn13': '978034552...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/38ad2956-8a45-5faf-b92e-36dde816b76b,2011-12-11
13,14,12,6,0,0,None,9780316097383,"Little, Brown & Company",A widow keeps the identity of the new man she ...,0.00,THE CHRISTMAS WEDDING,James Patterson and Richard DiLallo,by James Patterson and Richard DiLallo,,https://storage.googleapis.com/du-prd/books/im...,326.0,495.0,http://www.amazon.com/The-Christmas-Wedding-Ja...,,,,,,"[{'isbn10': '031609739X', 'isbn13': '978031609...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/a96c9b52-07cd-5c68-a013-e6f7eacb589e,2011-12-11
14,15,11,4,0,0,None,9781459273702,Harlequin,Former high school sweethearts fall tentativel...,0.00,UNFINISHED BUSINESS,Nora Roberts,by Nora Roberts,,https://storage.googleapis.com/du-prd/books/im...,128.0,202.0,http://www.amazon.com/Unfinished-Business-Silh...,,,,,,"[{'isbn10': '1459295366', 'isbn13': '978145929...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/f85ba5a4-2eeb-578d-ab96-9e98b6363627,2011-12-11
0,1,0,1,0,0,None,9780316194495,"Little, Brown & Company",Alex Cross investigates when the president’s c...,0.00,KILL ALEX CROSS,James Patterson,by James Patterson,,https://storage.googleapis.com/du-prd/books/im...,320.0,495.0,http://www.amazon.com/Kill-Alex-Cross-James-Pa...,,,,,,"[{'isbn10': '0316198730', 'isbn13': '978031619...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/70b3e460-3a2f-57d3-b226-635b98302918,2011-12-04
1,2,0,1,0,0,None,9781101548134,Penguin Group,"Pursuing a shoplifter, Kinsey Millhone discove...",0.00,V IS FOR VENGEANCE,Sue Grafton,by Sue Grafton,,https://storage.googleapis.com/du-prd/books/im...,327.0,495.0,http://www.amazon.com/V-Vengeance-Sue-Grafton/...,,,,https://www.nytimes.com/2011/11/20/books/revie...,,"[{'isbn10': '0399157867', 'isbn13': '978039915...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/69c3b73d-bebb-5a0b-ba93-9f81cd9ef375,2011-12-04
2,3,1,2,0,0,1451627289,9781451627282,Scribner,An English teacher travels back to 1958 by way...,0.00,11/22/63,Stephen King,by Stephen King,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,http://www.amazon.com/11-22-63-A-Novel/dp/1451...,,https://www.nytimes.com/2011/10/31/books/steph...,,https://www.nytimes.com/2011/11/13/books/revie...,,"[{'isbn10': '1

In [61]:
#and a column indicating which list these come from
k2 = 8250
weekly_fiction_list = ['Combined Print & E-Book Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

8250

In [62]:
#let's add nyt_list as a column
top_15_bestsellers['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
top_15_bestsellers.tail(20)

/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_54512/2497100196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_15_bestsellers['nyt_list'] = nyt_list_weekly_fiction


,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri,week,nyt_list
10,11,10,15,0,0,0307272117,9780307272119,Knopf Doubleday Publishing,A hacker and a journalist investigate the disa...,0.00,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,by Stieg Larsson,,https://storage.googleapis.com/du-prd/books/im...,277.0,495.0,http://www.amazon.com/Girl-Dragon-Tattoo-Stieg...,,https://www.nytimes.com/2008/09/30/books/30kak...,,https://www.nytimes.com/2008/09/14/books/revie...,,"[{'isbn10': '0307454541', 'isbn13': '978030745...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/61741728-91b7-59e6-b70e-61e4e41c4a1f,2011-12-11,Combined Print & E-Book Fiction
11,12,8,4,0,0,0425243214,9780425243213,Penguin Group,"A historic hotel in Boonsboro, Md., is getting...",0.00,THE NEXT ALWAYS,Nora Roberts,by Nora Roberts,,https://storage.googleapis.com/du-prd/books/im...,330.0,487.0,http://www.amazon.com/The-Next-Always-BoonsBor...,,,,,,"[{'isbn10': '0425243214', 'isbn13': '978042524...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/f4eb3f60-9612-50f9-a193-2880aaabf5f2,2011-12-11,Combined Print & E-Book Fiction
12,13,13,7,0,0,0345527704,9780345527707,Random House Publishing,A killer is after the bounty hunter Stephanie ...,0.00,SMOKIN’ SEVENTEEN,Janet Evanovich,by Janet Evanovich,,https://storage.googleapis.com/du-prd/books/im...,301.0,495.0,http://www.amazon.com/Smokin-Seventeen-Stephan...,,,,,,"[{'isbn10': '0345527682', 'isbn13': '978034552...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/38ad2956-8a45-5faf-b92e-36dde816b76b,2011-12-11,Combined Print & E-Book Fiction
13,14,12,6,0,0,None,9780316097383,"Little, Brown & Company",A widow keeps the identity of the new man she ...,0.00,THE CHRISTMAS WEDDING,James Patterson and Richard DiLallo,by James Patterson and Richard DiLallo,,https://storage.googleapis.com/du-prd/books/im...,326.0,495.0,http://www.amazon.com/The-Christmas-Wedding-Ja...,,,,,,"[{'isbn10': '031609739X', 'isbn13': '978031609...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/a96c9b52-07cd-5c68-a013-e6f7eacb589e,2011-12-11,Combined Print & E-Book Fiction
14,15,11,4,0,0,None,9781459273702,Harlequin,Former high school sweethearts fall tentativel...,0.00,UNFINISHED BUSINESS,Nora Roberts,by Nora Roberts,,https://storage.googleapis.com/du-prd/books/im...,128.0,202.0,http://www.amazon.com/Unfinished-Business-Silh...,,,,,,"[{'isbn10': '1459295366', 'isbn13': '978145929...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/f85ba5a4-2eeb-578d-ab96-9e98b6363627,2011-12-11,Combined Print & E-Book Fiction
0,1,0,1,0,0,None,9780316194495,"Little, Brown & Company",Alex Cross investigates when the president’s c...,0.00,KILL ALEX CROSS,James Patterson,by James Patterson,,https://storage.googleapis.com/du-prd/books/im...,320.0,495.0,http://www.amazon.com/Kill-Alex-Cross-James-Pa...,,,,,,"[{'isbn10': '0316198730', 'isbn13': '978031619...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/70b3e460-3a2f-57d3-b226-635b98302918,2011-12-04,Combined Print & E-Book Fiction
1,2,0,1,0,0,None,9781101548134,Penguin Group,"Pursuing a shoplifter, Kinsey Millhone discove...",0.00,V IS FOR VENGEANCE,Sue Grafton,by Sue Grafton,,https://storage.googleapis.com/du-prd/books/im...,327.0,495.0,http://www.amazon.com/V-Vengeance-Sue-Grafton/...,,,,https://www.nytimes.com/2011/11/20/books/revie...,,"[{'isbn10': '0399157867', 'isbn13': '978039915...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/69c3b73d-bebb-5a0b-ba93-9f81cd9ef375,2011-12-04,Combined Print & E-Book Fiction
2,3,1,2,0,0,1451627289,9781451627282,Scribner,An English teacher travels back to 1958 by way...,0.00,11/22/63,Stephen King,by Stephen Kin

In [64]:
#and save to another csv
top_15_bestsellers.to_csv("../data/top_15_fiction_bestsellers_since_dec_2011.csv", index=False)

In [57]:
#get other lists
r2 = requests.get('https://api.nytimes.com/svc/books/v3/lists/names.json?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH')

In [58]:
bs = r2.json()

In [59]:
print(bs)

{'status': 'OK', 'copyright': 'Copyright (c) 2022 The New York Times Company.  All Rights Reserved.', 'num_results': 59, 'results': [{'list_name': 'Combined Print and E-Book Fiction', 'display_name': 'Combined Print & E-Book Fiction', 'list_name_encoded': 'combined-print-and-e-book-fiction', 'oldest_published_date': '2011-02-13', 'newest_published_date': '2022-06-19', 'updated': 'WEEKLY'}, {'list_name': 'Combined Print and E-Book Nonfiction', 'display_name': 'Combined Print & E-Book Nonfiction', 'list_name_encoded': 'combined-print-and-e-book-nonfiction', 'oldest_published_date': '2011-02-13', 'newest_published_date': '2022-06-19', 'updated': 'WEEKLY'}, {'list_name': 'Hardcover Fiction', 'display_name': 'Hardcover Fiction', 'list_name_encoded': 'hardcover-fiction', 'oldest_published_date': '2008-06-08', 'newest_published_date': '2022-06-19', 'updated': 'WEEKLY'}, {'list_name': 'Hardcover Nonfiction', 'display_name': 'Hardcover Nonfiction', 'list_name_encoded': 'hardcover-nonfiction', '